# Recreating deeppavlov stuff using gensim+sklearn

In [2]:
# Import 
from gensim.test.utils import common_texts
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import numpy as np
import json

## Testing stuff

In [3]:
# Basic model
dataset = common_texts
dct = Dictionary(dataset)   # Set of avaliable text
corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format (i.e for each word in every sentece return "(index,nr times in sentence)""

model = TfidfModel(corpus)  # fit model
vector = model[corpus]   # apply model to the corpus document

## Our data

In [4]:
File = open("openvoc-keyword-spotting-research-datasets/smart-lights/metadata.json")
metaData_in = json.load(File)
N =  len(metaData_in)
keys = list(metaData_in.keys())

y_raw = []
X = []

for i in range(N):
    X.append(metaData_in.get(keys[i]).get('transcript').split(' '))
    y_raw.append(metaData_in.get(keys[i]).get('keywords'))
print(X[0])
print(y_raw[0])

['i', 'want', 'to', 'turn', 'off', 'the', 'lights', 'for', 'the', 'living', 'room']
['turn off', 'living room']


In [5]:
#keywords to user action
y = []
y_num = np.empty(N)
i = 0
for line in y_raw:
    for word in line:
        if word == 'turn off':
            y.append('SwitchLightOff')
            y_num[i] = 0
            break
        elif word == 'turn on':
            y.append('SwitchLightOn')
            y_num[i] = 1
            break
        elif word == 'increase':
            y.append('IncreaseBrightness')
            y_num[i] = 2
            break
        elif word == 'decrease':
            y.append('DecreaseBrightness')
            y_num[i] = 3
            break
        else:
            y.append('No class')
            y_num[i] = 4
            break
    i += 1

In [ ]:
"""
Add prerocessing section!
remove stop words, "steming+limitization"?
"""

In [6]:
# Create TD*IDF vector represenation
dct_X = Dictionary(X)
corpusBOW = [dct_X.doc2bow(line) for line in X]
model = TfidfModel(corpusBOW)
X_vec = model[corpusBOW]


In [11]:
def Tfidf2np(X,dct):
    """ 
    Converts gensim format to numpy array
    Input:
    X - TDidfModel vector (N x lenght("sentence"))
    dct - Dictionary object (lenght("unique words"))
    Output:
    X_np - N x length("unique words")
    """
    N_dict = len(dct)
    N_sent = len(X)
    X_np = np.zeros((N_sent,N_dict))
    i = 0
    for list in X:
        for word in list:
            X_np[i, word[0]] = word[1]
        i += 1
    return X_np

X_np = Tfidf2np(X_vec,dct_X)

In [12]:
# Split into training and test data
x_train, x_test, y_train, y_test = train_test_split(X_np,y_num,test_size = 0.5)

In [13]:
# Create LR model (add tuning parameters?)
cls = LogisticRegression()

In [14]:
cls.fit(x_train,y_train) 
score = cls.score(x_test,y_test)
print(score)

1.0
